In [1]:
import py_stringsimjoin as ssj
from py_stringsimjoin.debugmatcher.debug_matcher import *
from py_stringsimjoin.labeler.labeler import *
from py_stringsimjoin.active_learner.active_learner import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import py_stringmatching as sm
from py_stringsimjoin.sampler.sample import *
from py_stringsimjoin.feature.autofeaturegen import *
from py_stringsimjoin.feature.extractfeatures import *

In [2]:
ws=sm.WhitespaceTokenizer(return_set=True)
qg3=sm.QgramTokenizer(qval=3,return_set=True)

In [3]:
ldf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/dblp.csv')
rdf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/google_scholar.csv')

In [4]:
import re
ldf['cln_title'] = ldf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
rdf['cln_title'] = rdf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))

In [5]:
seed=pd.read_csv('/scratch/citseed.csv')

In [6]:
c=sample_pairs(ldf,rdf,'id','id','cln_title','cln_title',100000,25,seed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [7]:
len(c)

100004

In [8]:
labeled_c = label_table_using_gold(c, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_c['label'])

331

In [9]:
ft=get_features()
ft.drop('dice_ws', inplace=True)
ft.drop('cosine_ws', inplace=True)
ft.drop('overlap_ws', inplace=True)
ft.drop('overlap_coefficient_ws', inplace=True)

In [29]:
ft.ix['jaccard_ws']['tokenizer'] = qg3
fvs = extract_feature_vecs(c, 'l_id','r_id',ldf,rdf,'id','id','cln_title','cln_title',ft,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:22


In [30]:
len(fvs)

100004

In [31]:
lr=LogisticRegression()
al=ActiveLearner(lr,20,20,'/scratch/citgold.csv',seed)

In [32]:
lp = al.learn(fvs, '_id', 'l_id','r_id')

                              l_id          r_id  jaccard_ws
_id                                                         
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    1.000000
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.009804
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.009804
                              l_id          r_id  jaccard_ws  label
_id                                                                
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    1.000000      1
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.009804      0
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000      1
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.009804      0
Selecting next batch...
computing entropy
sorting
sampling
[0.69314718054137536, 0.69314718015166887, 0.69314718015166887, 0.69314718015166887, 0.69314717917956759, 0.69314717917956759, 0.69314717911831358, 0.69314717750629706

In [33]:
lp.head()

,l_id,r_id,jaccard_ws,label
_id,,,,
100000,conf/vldb/ChakrabartiDAR97,8IR1NrQ4zm8J,1.000000,1
100001,conf/vldb/ChakrabartiDAR97,DKUouEkK_b4J,0.009804,0
100002,conf/sigmod/LabrinidisR00,DKUouEkK_b4J,1.000000,1
100003,conf/sigmod/LabrinidisR00,8IR1NrQ4zm8J,0.009804,0
19882,conf/sigmod/HjaltasonS98,uQd6l3cuqlQJ,0.247059,0


In [34]:
lp.reset_index(level=0, inplace=True)
sum(lp['label'])

151

In [35]:
import py_entitymatching as em
em.set_property(lp,'key','_id')
em.set_property(ldf,'key','id')
em.set_property(rdf,'key','id')
em.set_ltable(lp,ldf)
em.set_rtable(lp,rdf)
em.set_fk_ltable(lp,'l_id')
em.set_fk_rtable(lp,'r_id')
splits = em.split_train_test(lp)
P=splits['train']
Q=splits['test']

In [36]:
len(P), len(Q)
sum(P['label']), sum(Q['label'])

(81, 70)

In [37]:
lr1=LogisticRegression()
dbg = debug_matcher(lr1, P, Q, 'jaccard_ws', 'label', 'l_id', 'r_id', ldf, rdf, 'id', 'id')

In [43]:
dbg['false_neg'][['l_cln_title', 'r_cln_title', 'jaccard_ws']]

,l_cln_title,r_cln_title,jaccard_ws
224,xsb as a deductive database,xsb as an efficient deductive database system,0.490196
143,effective change detection using sampling,effective change detection using sampling extended version,0.655172
303,disseminating updates on broadcast disks,disseminating updates on broadcast disks proceedings of the vldb conference,0.513158
272,qc trees an efficient summary structure for semantic olap,quotient cube and qc tree efficient summarizations for semantic olap,0.454545
307,fast algorithms for mining association rules in large databases,ramakrishnan srikant fast algorithms for mining association rules,0.500000
177,cache and query for wide area sensor databases,cache and query for wide area sensor network,0.636364
187,realm based spatial data types the rose algebra,realm based spatial data types the rose algebra the very large data base j vol 4 no 2,0.571429
70,the ins and outs and everthing in between of data warehousing,the ins and outs of data warehousing,0.606557
89,multimediaminer a system prototype for multimedia data mining,multimediaminer a system prototype for multimedia data mining proceedings of acm sigmod,0.646341
129,fast similarity search in the presence of noise scaling and translation in time series databases,hs and shim k 1995 fast similarity search in the presence of noise scaling and translation in,0.586777


In [39]:
dbg['stats']

{'f1': 0.796875, 'precision': 0.8793103448275862, 'recall': 0.7285714285714285}

In [40]:
-lr1.intercept_/lr1.coef_

array([[ 0.68120608]])

In [42]:
len(dbg['false_neg']), len(dbg['false_pos'])

(19, 7)